In [1]:
import graphlab

In [2]:
people = graphlab.SFrame('people_wiki.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to yy@ixi.ru and will expire on August 29, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1474293050.log


# Compare  top word count and tf-idf count

In [4]:
elton=people[people['name'] == 'Elton John']
elton

URI,name,text
<http://dbpedia.org/resource/Elton_John> ...,Elton John,sir elton hercules johncbe born reginald ken ...


In [9]:
elton['word_count'] = graphlab.text_analytics.count_words(elton['text'])

## Split word_count into two columns

In [10]:
elton_word_count_table = elton[['word_count']].stack('word_count', new_column_name=['word','count'])

In [13]:
elton_word_count_table.sort('count', ascending=False)

word,count
the,27
in,18
and,15
of,13
a,10
has,9
john,7
he,7
on,6
award,5


## Create TFIDF frame and count TFIDF top words

In [36]:
people['word_count'] = graphlab.text_analytics.count_words(people['text'])
tfidf = graphlab.text_analytics.tf_idf(people['word_count'])
people['tfidf'] = tfidf

In [18]:
elton = people[people['name']=="Elton John"]

In [20]:
elton[['tfidf']].stack('tfidf',new_column_name=['word','tfidf']).sort('tfidf',ascending=False)

word,tfidf
furnish,18.38947184
elton,17.48232027
billboard,17.3036809575
john,13.9393127924
songwriters,11.250406447
tonightcandle,10.9864953892
overallelton,10.9864953892
19702000,10.2933482087
fivedecade,10.2933482087
aids,10.262846934


# Now measure a distance

In [24]:
paul = people[people['name'] == 'Paul McCartney']
paul[['tfidf']].stack('tfidf',new_column_name=['word','tfidf']).sort('tfidf',ascending=False)

word,tfidf
mccartney,25.3301401563
lennon,18.9691688853
beatles,16.5050953672
starr,12.2533659697
solo,11.3545130372
lainemccartney,10.9864953892
musicmccartney,10.9864953892
harrison,10.6820969832
mbes,10.2933482087
wings,10.1622670823


In [25]:
victoria = people[people['name'] == 'Victoria Beckham']
victoria[['tfidf']].stack('tfidf',new_column_name=['word','tfidf']).sort('tfidf',ascending=False)

word,tfidf
beckham,56.9223550265
fashion,21.1652873531
spice,19.9280699021
victoria,15.7663610884
designer,15.6191472125
chartbeckham,10.9864953892
lowerpriced,10.9864953892
label,9.90513433711
clamour,9.88788310056
topmodel,9.60020102811


In [27]:
graphlab.distances.cosine(elton['tfidf'][0],paul['tfidf'][0])

0.8250310029221779

In [28]:
graphlab.distances.cosine(elton['tfidf'][0],victoria['tfidf'][0])

0.9567006376655429

# Calculate nearest neighbors with different input features and setting the distance metrics

## Create nearest neighbors model

In [40]:
knn_model_wordcount = graphlab.nearest_neighbors.create(people,features=['word_count'],label='name', distance='cosine')
knn_model_tfidf = graphlab.nearest_neighbors.create(people,features=['tfidf'],label='name', distance='cosine')

Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

In [41]:
knn_model_wordcount.query(elton)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 17.92ms      |

| Done         |         | 100         | 170.433ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Elton John,2.22044604925e-16,1
0,Cliff Richard,0.16142415259,2
0,Sandro Petrone,0.16822542751,3
0,Rod Stewart,0.168327165587,4
0,Malachi O'Doherty,0.177315545979,5


In [42]:
knn_model_tfidf.query(elton)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 9.241ms      |

| Done         |         | 100         | 273.547ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Elton John,-2.22044604925e-16,1
0,Rod Stewart,0.717219667893,2
0,George Michael,0.747600998969,3
0,Sting (musician),0.747671954431,4
0,Phil Collins,0.75119324879,5


In [44]:
knn_model_wordcount.query(victoria)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 7.13ms       |

| Done         |         | 100         | 169.845ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Victoria Beckham,-2.22044604925e-16,1
0,Mary Fitzgerald (artist),0.207307036115,2
0,Adrienne Corri,0.214509782788,3
0,Beverly Jane Fry,0.217466468741,4
0,Raman Mundair,0.217695474992,5


In [45]:
knn_model_tfidf.query(victoria)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 10.588ms     |

| Done         |         | 100         | 296.096ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Victoria Beckham,1.11022302463e-16,1
0,David Beckham,0.548169610263,2
0,Stephen Dow Beckham,0.784986706828,3
0,Mel B,0.809585523409,4
0,Caroline Rush,0.819826422919,5
